In [ ]:
import pygame
import pyaudio
import wave
import speech_recognition as sr
from tkinter import *
from tkinter import messagebox
import librosa
import numpy as np
from tensorflow.keras.models import load_model

def make_spec(file, flip = False, ps = False, st = 4):
    sig, rate = librosa.load(file, sr = 16000)
    if len(sig) < 16000: # 1초 미만의 오디오를 0으로 pad 함수를 이용하여 변환
        sig = np.pad(sig, (0,16000-len(sig)), 'linear_ramp')
    if ps:
        # 소리의 원래 피치를 올리거나 내리며 4단으로 변경
        sig = librosa.effects.pitch_shift(sig, rate, st)
        
    # 진폭 분광 프로그램을 dB 단위 분광 프로그램으로 변환
    D = librosa.amplitude_to_db(librosa.stft(sig[:16000], n_fft = 512, 
                                             hop_length = 128, 
                                             center = False), ref = np.max)
    # mel-scaled spectrogram으로 변환
    S = librosa.feature.melspectrogram(S=D, n_mels = 85).T
    if flip:
        # 배열을 위아래 방향을 전환
        S = np.flipud(S)
    return S.astype(np.float32)

def predict(file):
    import librosa
    from tensorflow.keras.models import load_model
    model = load_model('my_model.h5')
    classes = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'backward', 'forward','house','silence', 'unknown']

    command = make_spec(file)
    command = np.expand_dims(command, -1)+1.3
    command = np.expand_dims(command, 0)
    result1 = model.predict(command, batch_size=16, verbose=1)
    
    return classes[result1.argmax()]

#voice-record 클래스 
class microphone():

    def record(self):
        FORMAT = pyaudio.paInt16
        CHANNELS = 2
        RATE = 44100
        CHUNK = 1
        RECORD_SECONDS = 2 #녹음할 시간 
        WAVE_OUTPUT_FILENAME = "command.wav"
 
        audio = pyaudio.PyAudio()
 
        # start Recording
        stream = audio.open(format=FORMAT, channels=CHANNELS,
                rate=RATE, input=True,
                frames_per_buffer=CHUNK)
        print ("recording...")
        frames = []
 
        for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
            data = stream.read(CHUNK)
            frames.append(data)
        print ("finished recording")
 
        # stop Recording
        stream.stop_stream()
        stream.close()
        audio.terminate()
 
        waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
        waveFile.setnchannels(CHANNELS)
        waveFile.setsampwidth(audio.get_sample_size(FORMAT))
        waveFile.setframerate(RATE)
        waveFile.writeframes(b''.join(frames))
        waveFile.close() 
        
class Game():

    def start(self):       
        check_start=0
        homearr=['f','right','right','right','right','right','right','right'
                ,'right','right','right','right','right','right','right','right','right'
                ,'f','f','f','f','f','f','f','f','f','f']
        home_index=0
        home_check=0
        pygame.init()

        display_width = 800
        display_height = 600

        gameDisplay = pygame.display.set_mode((display_width,display_height))
        pygame.display.set_caption('Speech Car')

        black = (0,0,0)
        white = (255,255,255)
        clock = pygame.time.Clock()
        crashed = False

        #font, text 
        font = pygame.font.SysFont("comicsansms", 30)
        text = font.render("START!!!", True, (0, 128, 0))

        font1 = pygame.font.SysFont("comicsansms", 30)
        text1 = font.render("COMMAND", True, (0, 128, 0))

        font2 = pygame.font.SysFont("comicsansms", 20)
        text_up = font.render("U", True, (0, 128, 0))
        text_down = font.render("D", True, (0, 128, 0))
        text_left = font.render("L", True, (0, 128, 0))
        text_right = font.render("R", True, (0, 128, 0))
        
        text_ing = font.render("...ing", True, (0, 128, 0))
        text_again = font.render("speak again!", True, (0, 128, 0))


        #background img
        bg=pygame.image.load('road2.png')
        bg = pygame.transform.scale(bg, (display_width, display_height))
        #image
        carImg = pygame.image.load('car.png')
        carImg = pygame.transform.scale(carImg, (35, 35))

        homeImg = pygame.image.load('home1.png')
        homeImg = pygame.transform.scale(homeImg, (80, 80))

        #button
        button = pygame.Rect(20, display_height-45, 170, 50) 
        button2 = pygame.Rect(200, display_height-45, 170, 50) 

        #up down left right button
        btn_up = pygame.Rect(420, display_height-40, 40, 40) 
        btn_down = pygame.Rect(470, display_height-40, 40, 40) 
        btn_left = pygame.Rect(520, display_height-40, 40, 40) 
        btn_right = pygame.Rect(570, display_height-40, 40, 40) 

        def text_draw(x,y,gameDisplay,text,text1,text_up,text_down,text_left,text_right,text_alert):
            gameDisplay.blit(text,(40,555))
            gameDisplay.blit(text1,(205,555))

            gameDisplay.blit(text_up,(430,555))
            gameDisplay.blit(text_down,(480,555))
            gameDisplay.blit(text_left,(530,555))
            gameDisplay.blit(text_right,(580,555))
            
            gameDisplay.blit(text_alert,(620,555))

        def car(x,y,gameDisplay,carImg,homeImg):
            gameDisplay.blit(carImg, (x,y))
            gameDisplay.blit(homeImg, (668,5))

        def background(x,y,gameDisplay,bg):
            gameDisplay.blit(bg, (x,y))

        #이동 시작 좌표 
        x=40
        y=500
        x_change = 0
        y_change = 0
        car_speed = 0
        move_size=40
        er=0
        while not crashed:
            if home_check==1:
                if homearr[home_index]=='b' :
                    y_change=move_size
                elif homearr[home_index]=='f' :
                    y_change=-move_size
                elif homearr[home_index]=="left" :
                    x_change=-move_size
                elif homearr[home_index]=="right" :
                    x_change=move_size
                    
                home_index=home_index+1
            else :
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        sys.exit()
                        crashed = True
                    #button event
                    if event.type==pygame.MOUSEBUTTONDOWN:
                        mouse_pos=event.pos 

                        #start button - button
                        if button.collidepoint(mouse_pos):
                            #처음부터 
                            x=40
                            y=500
                            home_index=0
                            home_check=0
                            check_start=0

                        #command button - button2
                        if button2.collidepoint(mouse_pos):
                            check_start=1
                             #voice record
                            mic=microphone()
                            mic.record()

                            #str_command 받아 오기 !! model-predict()!!!!
                            filename='command.wav'
                            
                            str_command = predict(filename)

                            print(str_command)
                            str_command.upper().lower()
                            #command로 좌표 설정 
                            if str_command[0]=='b' :
                                er=0
                                y_change=move_size
                            elif str_command[0]=='f' :
                                er=0
                                y_change=-move_size
                            elif str_command=="left" :
                                er=0
                                x_change=-move_size
                            elif str_command=="right" :
                                er=0
                                x_change=move_size
                            elif str_command=="error" :
                                er=1
                                #break
                            elif str_command=="house":
                                if(check_start==1):
                                    home_check=1
                      
                        #임시 button listener
                        if btn_up.collidepoint(mouse_pos):
                                y_change=-move_size
                        if btn_down.collidepoint(mouse_pos):
                                y_change=move_size
                        if btn_left.collidepoint(mouse_pos):
                                x_change=-move_size
                        if btn_right.collidepoint(mouse_pos):
                                x_change=move_size
            tempx=x
            tempy=y
            x += x_change
            y += y_change

            #범위 예외 처리 
            if(x<40 or y<0 or x>720 or y >500): #바깥 블럭 예외처리 
                x=tempx
                y=tempy
                Tk().wm_withdraw() #to hide the main window
                messagebox.showinfo('SpeechCar','범위를 벗어났습니다')
            if((x>=160 and y>=500)or(x>=120 and x<=520 and y>=180 and y<=260)):
                x=tempx
                y=tempy
                Tk().wm_withdraw() #to hide the main window
                messagebox.showinfo('SpeechCar','범위를 벗어났습니다')
            if((x>=160 and x<=400 and y<=20)or(x>=560 and x<=640 and y<=420)):
                x=tempx
                y=tempy
                Tk().wm_withdraw() #to hide the main window
                messagebox.showinfo('SpeechCar','범위를 벗어났습니다')
            if(x>=680 and x<=720 and y<=60):
                x=tempx
                y=tempy
                Tk().wm_withdraw() #to hide the main window
                messagebox.showinfo('SpeechCar','도착했습니다!!!!')
                x=40
                y=500
                home_check=0
                check_start=0
                home_index=0

            gameDisplay.fill(white)
            background(0,0,gameDisplay,bg)
            car(x,y,gameDisplay,carImg,homeImg)

            #button draw  
            pygame.draw.rect(gameDisplay, [204, 204, 204], button)
            pygame.draw.rect(gameDisplay, [204, 204, 204], button2)

            #임시 button draw
            pygame.draw.rect(gameDisplay, [204, 204, 204], btn_up)
            pygame.draw.rect(gameDisplay, [204, 204, 204], btn_down)
            pygame.draw.rect(gameDisplay, [204, 204, 204], btn_left)
            pygame.draw.rect(gameDisplay, [204, 204, 204], btn_right)
            
            if(er==1):
                text_draw(x,y,gameDisplay,text,text1,text_up,text_down,text_left,text_right,text_again)
            else :
                text_draw(x,y,gameDisplay,text,text1,text_up,text_down,text_left,text_right,text_ing)
            

            #print(x,y)
            pygame.display.update()
            clock.tick(60)

            #이동 초기화 
            x_change=0
            y_change=0
           

    pygame.quit()

g=Game()
g.start()

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html
